Script per generare un dataset per il contrastive learning che contiene 3 stringhe per ogni riga:
- Un tweet
- Un testo che contiene lo stesso ade e meddra (positive)
- Un testo che contiene un altro ade e meddra (negative)

In [25]:
import pandas as pd
from nltk.corpus import wordnet2021 as wd
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('wordnet2021')

from simcse import SimCSE
import random
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet2021 to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet2021 is already up-to-date!


In [26]:
df1 = pd.read_csv('./datasets/cadec_meddra.csv', sep=",")
df1 = df1.drop('full_text', axis=1)
df2 = pd.read_csv('./datasets/smm4h20_meddra.csv', sep=",")
df = pd.concat([df1, df2])

Dividiamo i dati in training set e evaluation set.

In [27]:
train, test = train_test_split(df, test_size=0.8)

In [28]:
if 'model' not in locals():
  model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

Prendiamo ogni riga del dataset di training per trovare un esempio positive e un esempio negative.
- Un esempio positivo è un testo che contiene lo stesso ade mappato nello stesso meddra
- Un esempio negativo è un testo che che non contiene l'ade e il meddra

In [29]:
ades = train['ade']
meddras = train['meddra']
positives = []
negatives = []

In [48]:
# Calcolo dei positives e dei negatives
for index, (text, ade, meddra, _) in train.iterrows():
  same_ade_meddra_pair = df.loc[(df['ade'] == ade) & (df['meddra'] == meddra)]
  # Se non ci sono righe con lo stesso paio ade, meddra allora l'esempio positivo è il meddra
  positive: str = meddra
  if not same_ade_meddra_pair.empty:
    positive = same_ade_meddra_pair.sample(n=1)['text'].values[0]

  # Se non ci sono esempi negativi allora il negative è un testo con meddra diverso
  negative: str = train.loc[train['meddra'] != meddra].sample(n=1)['text']

  positives.append(positive)
  negatives.append(negative)

print(positives[0])
print(negatives[0])

674    coming off of cymbalta has been the most exhau...
Name: text, dtype: object
943    arm stiffness.
Name: text, dtype: object


Salviamo il dataset

In [ ]:
to_save = pd.DataFrame({
  'text' : train['text'],
  'positive' : positives,
  'negative' : negatives,
  'ade': train['ade'],
  'meddra': train['meddra']
})

to_save.to_csv('training/training_set.csv')
test.to_csv('training/test_set.csv')